In [1]:
import torch
import yolov5
import cv2
import requests
from PIL import Image
from io import BytesIO
import ollama

In [5]:
# Load video data
cap = cv2.VideoCapture("dashcamtest.mov")

# Load YOLOv5 model
model = yolov5.load("yolov5s.pt")

In [9]:
# Define grid dimensions (e.g., 3x3 grid)
grid_rows = 3
grid_cols = 3

# Define class labels (example for COCO dataset)
class_labels = model.names

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Get frame dimensions
    frame_height, frame_width = frame.shape[:2]

    # Calculate grid cell size
    cell_width = frame_width // grid_cols
    cell_height = frame_height // grid_rows

    # Convert the frame to a PIL image
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame_rgb)

    # Run the model on the frame
    results = model(pil_image)

    # Draw the bounding boxes and check grid cells
    for result in results.xyxy[0]:
        x1, y1, x2, y2, conf, class_id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

        # Determine which grid cell the object is in
        cell_x = x1 // cell_width
        cell_y = y1 // cell_height

        # Get the class label
        class_label = class_labels[int(class_id)]

        # Print a statement when an object is detected in a grid cell
        print("Object detected in grid cell ({}, {}): {}".format(cell_x, cell_y, class_label))
        # response = ollama.chat(
        #     model='llama3.2',
        #     messages=[{
        #         'role': 'user',
        #         'content': "Object detected in grid cell ({}, {}): {}. What should I do to avoid it?".format(cell_x, cell_y, class_label),
        #     }],
        # )

    # Display the frame
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (0, 1): traffic light
Object detected in grid cell (0, 1): truck
Object detected in grid cell (0, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (0, 1): traffic light
Object detected in grid cell (0, 1): truck
Object detected in grid cell (0, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (0, 1): traffic light
Object detected in grid cell (0, 1): truck
Object detected in gr

/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): traffic light
Object detected in grid cell (0, 1): car
Object detected in grid cell (0, 1): truck
Object detected in grid cell (0, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): traffic light
Object detected in grid cell (0, 1): car
Object detected in grid cell (0, 1): truck
Object detected in grid cell (0, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): traffic light
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): truck
Object detected in gr

/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
2024-12-02 17:41:29.322 Python[70933:12369446] Warning: Wi

...

Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): car
Object detected 

/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIow

Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (0, 1): car
Object detected in grid cell (1, 1): car
Object detected in grid cell (1, 1): car


/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/Users/herdt/Library/CloudStorage/OneDrive-UniversityofIowa/Year 4/Senior Design/Code/BikeBuddy/.venv/lib/python3.12/site-packages/yolov5/models/common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


KeyboardInterrupt: 